<a href="https://colab.research.google.com/github/himasree33/deepfake-using-MesoNet/blob/main/Copy_of_finalmesonetml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install required packages

In [ ]:

# Note: TensorFlow 2.7 is used because the repo was tested with it.
!pip install -q tensorflow==2.7.0
!pip install -q opencv-python-headless==4.5.5.64
!pip install -q scikit-learn
!pip install -q imageio-ffmpeg
!pip install -q pillow
!pip install -q h5py
!pip install -q matplotlib


ERROR: Could not find a version that satisfies the requirement tensorflow==2.7.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.7.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 4.5.5.64 which is incompatible.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 4.5.5.64 which is incompatible.


#Clone MesoNet repo

In [ ]:

import os
%cd /content

if not os.path.exists('/content/MesoNet'):
    !git clone https://github.com/DariusAf/MesoNet.git
else:
    print("MesoNet already cloned.")

print("\nTop-level files in /content/MesoNet:")
print(sorted(os.listdir('/content/MesoNet')))


/content
Cloning into 'MesoNet'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 82 (delta 29), reused 22 (delta 22), pack-reused 47 (from 3)
Receiving objects: 100% (82/82), 494.53 KiB | 4.85 MiB/s, done.
Resolving deltas: 100% (32/32), done.

Top-level files in /content/MesoNet:
['.git', '.gitignore', 'LICENSE', 'NOTICE', 'README.md', 'classifiers.py', 'example.py', 'pipeline.py', 'test_images', 'weights']


#Check for expected weights

In [ ]:

import os

weights_dir = '/content/MesoNet/weights'
test_images_dir = '/content/MesoNet/test_images'
test_videos_dir = '/content/MesoNet/test_videos'

print("Weights folder exists:", os.path.exists(weights_dir))
if os.path.exists(weights_dir):
    print("Weights found:", sorted(os.listdir(weights_dir)))
else:
    print("No weights found. Put your .h5 files in /content/MesoNet/weights/")

print("test_images folder exists:", os.path.exists(test_images_dir))
print("test_videos folder exists:", os.path.exists(test_videos_dir))


Weights folder exists: True
Weights found: ['Meso4_DF.h5', 'Meso4_F2F.h5', 'MesoInception_DF.h5', 'MesoInception_F2F.h5']
test_images folder exists: True
test_videos folder exists: False


#import classifiers/pipeline

In [ ]:

import sys, os
repo_path = '/content/MesoNet'
if repo_path not in sys.path:
    sys.path.append(repo_path)
print("Added to sys.path:", repo_path in sys.path)

# Quick import test (safe)
try:
    from classifiers import Meso4
    print("Imported Meso4 from classifiers.py ")
except Exception as e:
    print("Warning: couldn't import Meso4 yet. Error:\n", e)
    print("Check that /content/MesoNet/classifiers.py exists and sys.path includes the repo.")


Added to sys.path: True
Imported Meso4 from classifiers.py 


In [ ]:
# Cell 5: Write meso_utils.py with helper functions for image/video prediction
utils_code = r"""
import os, cv2, numpy as np
from tensorflow.keras.preprocessing import image

IMG_TARGET = (256, 256)

def load_img_array(img_path, target_size=IMG_TARGET):
    # loads image and returns array in [0,1] range with shape (1, h, w, c)
    img = image.load_img(img_path, target_size=target_size)
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = arr.astype('float32') / 255.0
    return arr

def predict_image(model, img_path):
    x = load_img_array(img_path)
    pred = model.predict(x)
    return float(np.squeeze(pred))

def predict_folder(model, folder_path, exts=('jpg','jpeg','png')):
    files = []
    for root, _, names in os.walk(folder_path):
        for n in names:
            if n.split('.')[-1].lower() in exts:
                files.append(os.path.join(root, n))
    files = sorted(files)
    preds = {}
    for p in files:
        try:
            preds[p] = predict_image(model, p)
        except Exception as e:
            preds[p] = None
    return preds

def frames_from_video(video_path, every_n_frames=10, max_frames=300, target_size=IMG_TARGET):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return []
    frames = []
    idx = 0
    grabbed = True
    while grabbed and len(frames) < max_frames:
        grabbed, frame = cap.read()
        if not grabbed:
            break
        if idx % every_n_frames == 0:
            # BGR -> RGB
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb = cv2.resize(rgb, target_size)
            frames.append(rgb)
        idx += 1
    cap.release()
    return frames

def predict_video(model, video_path, every_n_frames=10):
    frames = frames_from_video(video_path, every_n_frames=every_n_frames)
    if len(frames) == 0:
        return None, []
    arr = np.array(frames, dtype=np.float32) / 255.0
    preds = model.predict(arr, batch_size=32)
    preds = np.squeeze(preds)
    # ensure preds is 1-D array
    if preds.ndim == 0:
        preds = np.array([float(preds)])
    agg = float(np.mean(preds))
    return agg, preds.tolist()
"""
open('/content/meso_utils.py','w').write(utils_code)
print("Wrote /content/meso_utils.py")


Wrote /content/meso_utils.py


#Instantiate Meso4 and load weights

In [ ]:

import os
from meso_utils import predict_image, predict_folder, predict_video, load_img_array
from classifiers import Meso4

weights_path = '/content/MesoNet/weights/Meso4_DF.h5'  # change if needed

classifier = Meso4()
if os.path.exists(weights_path):
    try:
        classifier.load(weights_path)
        print("Loaded weights:", weights_path)
    except Exception as e:
        print("Error loading weights:", e)
else:
    print("Weights not found at", weights_path, "\nPlease put the .h5 file there or change weights_path variable.")


Loaded weights: /content/MesoNet/weights/Meso4_DF.h5


In [ ]:
# Cell 7: Single-image test
# Replace img_path with an actual image inside /content/MesoNet/test_images or upload one.
img_path = '/content/MesoNet/test_images/sample.jpg'  # <-- change this to an image that exists

import os
if not os.path.exists(img_path):
    # try pick first image from test_images automatically
    test_folder = '/content/MesoNet/test_images'
    found = None
    for root, _, files in os.walk(test_folder):
        for f in files:
            if f.lower().endswith(('.jpg','.jpeg','.png')):
                found = os.path.join(root, f)
                break
        if found:
            break
    if found:
        img_path = found
        print("Auto-picked image:", img_path)
    else:
        raise FileNotFoundError("No images found in /content/MesoNet/test_images. Upload or change img_path.")

score = predict_image(classifier, img_path)
print("Predicted fake-score (higher -> more likely fake):", score)


Auto-picked image: /content/MesoNet/test_images/real/real00240.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Predicted fake-score (higher -> more likely fake): 0.9897475838661194


In [ ]:
# Cell 8: Batch predict folder
import csv
folder = '/content/MesoNet/test_images'   # set your folder
out_csv = '/content/predictions_images.csv'

preds = predict_folder(classifier, folder)
with open(out_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['path','score'])
    for p, s in preds.items():
        writer.writerow([p, s])
print("Saved predictions to", out_csv, " - rows:", len(preds))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Saved predictions to /content/predictions_images.csv  - rows: 4


In [ ]:
# Cell 9: Batch predict videos
video_folder = '/content/MesoNet/test_videos'   # change if necessary
out_json = '/content/video_predictions.json'
import json, os

results = {}
for root, _, files in os.walk(video_folder):
    for fn in sorted(files):
        if fn.lower().endswith(('.mp4','.avi','.mov','.mkv')):
            path = os.path.join(root, fn)
            print("Processing:", path)
            agg, frame_preds = predict_video(classifier, path, every_n_frames=10)
            results[fn] = {'path': path, 'score': agg, 'n_frames': len(frame_preds)}
            print(" -> score:", agg, "frames:", len(frame_preds))

with open(out_json, 'w') as f:
    json.dump(results, f, indent=2)
print("Saved video predictions to", out_json)


Saved video predictions to /content/video_predictions.json


In [ ]:
# Cell 10: Evaluate predictions given a labels CSV
# labels CSV format: path,label  where label is 0 (real) or 1 (fake)
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

labels_csv = '/content/MesoNet/labels.csv'  # create this file: path,label
pred_csv = '/content/predictions_images.csv'  # from Cell 8

if not os.path.exists(labels_csv):
    print("labels.csv not found at", labels_csv, " - create one with columns: path,label")
else:
    df_labels = pd.read_csv(labels_csv)
    pred_df = pd.read_csv(pred_csv)
    pred_map = dict(zip(pred_df['path'], pred_df['score']))

    y_true = []
    y_score = []
    missing = 0
    for _, row in df_labels.iterrows():
        p = row['path']
        if p not in pred_map:
            missing += 1
            continue
        y_true.append(int(row['label']))
        y_score.append(pred_map[p])

    print("Missing entries:", missing, "Total evaluated:", len(y_true))
    if len(y_true) > 0:
        y_pred = [1 if s >= 0.5 else 0 for s in y_score]
        acc = accuracy_score(y_true, y_pred)
        prec, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
        try:
            auc = roc_auc_score(y_true, y_score)
        except:
            auc = None
        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc}")
        if auc is not None:
            fpr, tpr, _ = roc_curve(y_true, y_score)
            plt.plot(fpr, tpr)
            plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC'); plt.grid(True)
            plt.show()


labels.csv not found at /content/MesoNet/labels.csv  - create one with columns: path,label


In [ ]:
import pandas as pd

df = pd.read_csv('/content/predictions_images.csv')  # or the file you saved
df['prediction'] = df['score'].apply(lambda s: 'FAKE' if s <= 0.5 else 'REAL')
df['confidence'] = df['score'].apply(lambda s: round(s if s >= 0.5 else 1-s, 3))

display(df)


,path,score,prediction,confidence
0,/content/MesoNet/test_images/df/dp1.jpg,0.059061,FAKE,0.941
1,/content/MesoNet/test_images/df/dp2.jpg,0.992756,REAL,0.993
2,/content/MesoNet/test_images/real/dp3.jpg,0.999998,REAL,1.000
3,/content/MesoNet/test_images/real/dp4.jpg,0.078441,FAKE,0.922
